# Libraries


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import os
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import optimizers
from keras.initializers import Constant
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU, PReLU
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import tensorflow as tf


/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning:

Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons



# Parameters to be used in the convolutional network

In [2]:
IMAGE_PATH = '../input/chinese-mnist/data/data/'
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_CHANNELS = 1
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
BATCH_SIZE = 32
NO_EPOCHS = 50
PATIENCE = 5
VERBOSE = 1

# Data

In [3]:
data_df = pd.read_csv('../input/chinese-mnist/chinese_mnist.csv')
print(data_df.shape) 
data_df.sample(100).head(4)

(15000, 5)


,suite_id,sample_id,code,value,character
3474,51,4,13,1000,千
14869,87,9,9,8,八
12176,24,6,7,6,六
12166,23,6,7,6,六


# Checking the amount of images stored

In [4]:
image_files = list(os.listdir(IMAGE_PATH))
print("Number of image files: {}".format(len(image_files)))

Number of image files: 15000


# Adding the full name of the image

In [5]:
def create_file_name(x):
    
    file_name = f"input_{x[0]}_{x[1]}_{x[2]}.jpg"
    return file_name

In [6]:
data_df["file"] = data_df.apply(create_file_name, axis=1)
data_df.head()

,suite_id,sample_id,code,value,character,file
0,1,1,10,9,九,input_1_1_10.jpg
1,1,10,10,9,九,input_1_10_10.jpg
2,1,2,10,9,九,input_1_2_10.jpg
3,1,3,10,9,九,input_1_3_10.jpg
4,1,4,10,9,九,input_1_4_10.jpg


# Checking the amount of images


In [7]:
file_names = list(data_df['file'])
print("Matching image names: {}".format(len(set(file_names).intersection(image_files))))

Matching image names: 15000


# Checking the dimension of the images

In [8]:
def read_image_sizes(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    return list(image.shape)

In [9]:
tqdm.pandas()
m = np.stack(data_df['file'].progress_apply(read_image_sizes))
df = pd.DataFrame(m,columns=['w','h'])
data_df = pd.concat([data_df,df],axis=1, sort=False)
data_df.head()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

100%|██████████| 15000/15000 [00:30<00:00, 497.64it/s]


,suite_id,sample_id,code,value,character,file,w,h
0,1,1,10,9,九,input_1_1_10.jpg,64,64
1,1,10,10,9,九,input_1_10_10.jpg,64,64
2,1,2,10,9,九,input_1_2_10.jpg,64,64
3,1,3,10,9,九,input_1_3_10.jpg,64,64
4,1,4,10,9,九,input_1_4_10.jpg,64,64


# Checking samples and sub-samples of the data

In [10]:
print(f"Number of suites: {data_df.suite_id.nunique()}")
print(f"Samples: {data_df.sample_id.unique()}")

Number of suites: 100
Samples: [ 1 10  2  3  4  5  6  7  8  9]


# Creating datasets to build the network

In [11]:
train_df, test_df = train_test_split(data_df, 
                                     test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=data_df["code"].values)

In [12]:
train_df, val_df = train_test_split(train_df, 
                                    test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=train_df["code"].values)

In [13]:
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))
print("Val   set rows: {}".format(val_df.shape[0]))

Train set rows: 9600
Test  set rows: 3000
Val   set rows: 2400


# Encoding images to create network

In [14]:
def read_image(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT, 1), mode='reflect')
    return image[:,:,:]

In [15]:
def categories_encoder(dataset, var='character'):
    X = np.stack(dataset['file'].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

In [16]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

# Creating network topology, inspired by [Xiaoa (2017)](https://arxiv.org/pdf/1702.07975.pdf)

- the architecture is composed of 7 layers of convolution
- A hidden layer of 1024 neurons
- as an activation function PRelu is used
- It was not necessary to use a pruning process, as used in the paper




In [17]:
model=Sequential()

model.add(Conv2D(filters = 64, kernel_size=(3,3), input_shape=(64, 64, 1), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 128, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 160, kernel_size=(3,3),  padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 256, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(Conv2D(filters = 256, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 384, kernel_size=(3,3),  padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(Conv2D(filters = 384, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        640       
_________________________________________________________________
p_re_lu (PReLU)              (None, 64, 64, 64)        262144    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 32, 32, 128)       131072    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 160)       1

# Training the neural network

In [18]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.99 ** (x+NO_EPOCHS))
earlystopper = EarlyStopping(monitor='loss', patience=PATIENCE, verbose=VERBOSE)
checkpointer = ModelCheckpoint('best_model.h5',
                                monitor='val_accuracy',
                                verbose=VERBOSE,
                                save_best_only=True,
                                save_weights_only=True)

In [19]:
train_model  = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val),
                  callbacks=[earlystopper, checkpointer, annealer])

Epoch 1/50
297/300 [============================>.] - ETA: 0s - loss: 1.1066 - accuracy: 0.6031
Epoch 00001: val_accuracy improved from -inf to 0.91833, saving model to best_model.h5
300/300 [==============================] - 5s 17ms/step - loss: 1.0989 - accuracy: 0.6062 - val_loss: 0.2560 - val_accuracy: 0.9183
Epoch 2/50
300/300 [==============================] - ETA: 0s - loss: 0.1735 - accuracy: 0.9472
Epoch 00002: val_accuracy improved from 0.91833 to 0.97375, saving model to best_model.h5
300/300 [==============================] - 5s 17ms/step - loss: 0.1735 - accuracy: 0.9472 - val_loss: 0.0914 - val_accuracy: 0.9737
Epoch 3/50
300/300 [==============================] - ETA: 0s - loss: 0.1561 - accuracy: 0.9564
Epoch 00003: val_accuracy improved from 0.97375 to 0.98125, saving model to best_model.h5
300/300 [==============================] - 5s 16ms/step - loss: 0.1561 - accuracy: 0.9564 - val_loss: 0.0654 - val_accuracy: 0.9812
Epoch 4/50
299/300 [============================>

# Visualizing the evolution of loss and accuracy

In [20]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    #define the traces
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
    fig = tools.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    #add traces to the figure
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    #set the layout for the figure
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])
    #plot
    iplot(fig, filename='accuracy-loss')

plot_accuracy_and_loss(train_model)

# Reviewing the metrics in the validation and test sets

In [21]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0813622772693634
Test accuracy: 0.9903333187103271


In [22]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

Val loss: 0.064023457467556
Val accuracy: 0.9908333420753479


In [23]:
def test_accuracy_report(model):
    predicted = model.predict(X_test)
    test_predicted = np.argmax(predicted, axis=1)
    test_truth = np.argmax(y_test.values, axis=1)
    print(metrics.classification_report(test_truth, test_predicted, target_names=y_test.columns)) 
    test_res = model.evaluate(X_test, y_test.values, verbose=0)
    print('Loss function: %s, accuracy:' % test_res[0], test_res[1])

In [24]:
model_optimal = model
model_optimal.load_weights('best_model.h5')
score = model_optimal.evaluate(X_test, y_test, verbose=0)
print(f'Best validation loss: {score[0]}, accuracy: {score[1]}')

test_accuracy_report(model_optimal)

Best validation loss: 0.034993987530469894, accuracy: 0.9909999966621399
              precision    recall  f1-score   support

           一       1.00      1.00      1.00       200
           七       0.98      1.00      0.99       200
           万       1.00      1.00      1.00       200
           三       0.97      0.96      0.97       200
           九       1.00      0.96      0.98       200
           二       0.97      1.00      0.98       200
           五       1.00      0.97      0.98       200
           亿       0.99      1.00      0.99       200
           八       1.00      1.00      1.00       200
           六       1.00      1.00      1.00       200
           十       0.99      0.99      0.99       200
           千       0.99      0.98      0.99       200
           四       1.00      1.00      1.00       200
           百       1.00      0.99      1.00       200
           零       1.00      1.00      1.00       200

    accuracy                           0.99      3000
   macr

# Conclusions

- The model shows no signs of being over-fitting or under-fitting
- Metrics of more than 99% accuracy were obtained in both the test data and the validation data
- The architecture of the network is not exactly the same as the paper used but it has the same amount of layers.
- In the table above we can notice the accuracy per element, revealing that the element with less accuracy has a value of 0.97
- These results confirm the efficiency of the study done by Xiaoa